In [65]:
from ajk_utils.parse_fasta import parse_fasta
from ajk_utils.reverse_complement import reverse_complement
from ajk_utils.orf_finder import get_orfs
from ajk_utils.translate_genes import nuc_to_pro

In [66]:
def get_genes(seq, min_bp, nested):
    # Find forward strand orfs
    sequence = parse_fasta(seq)
    orfs_nested = get_orfs(sequence, min_bp, nested)
    
    # Create Reverse Complement strand
    comp_base = {'A': 'T', 'T': 'A', 'G': 'C', 'C':'G'}
    seq_list = list(sequence)
    seq_list.reverse()
    reverse = ''.join(seq_list)
    r_comp = ''.join([comp_base[i] for i in reverse])
    
    # Find reverse strand orfs
    orfs = get_orfs(r_comp, min_bp, nested)
    comp_orfs_nested = [((len(r_comp) + 1 - i[1]), (len(r_comp) + 1 - i[0]), '-') 
                for i in orfs]
    comp_orfs_nested.sort(key=lambda gene: (gene[0], gene[1], gene[2]))
    
    #Merge and sort both strands
    gene_list = [(i[0], i[1], '+') for i in orfs_nested]
    gene_list.extend(comp_orfs_nested)
    gene_list.sort(key=lambda gene: (gene[0], gene[1], gene[2]))
    
    return gene_list
    
    

In [86]:
def get_proteins(genes_list):
    database = [0]*len(genes_list)
    sequence = parse_fasta('sars_cov2.fasta')

    start_i = 0
    stop_i = 0

    for i in range(len(sars2_genes)):
        start_i = sars2_genes[i][0] - 1
        stop_i = sars2_genes[i][1] 
        if sars2_genes[i][2] == '-':
            subsequence = reverse_complement(sequence[start_i:stop_i])
        else:
            subsequence = sequence[start_i:stop_i] 
    
    
        protein = nuc_to_pro(subsequence)
        
        database[i] = [start_i + 1, stop_i, sars2_genes[i][2], protein]
        
    return database

In [87]:
sars2_genes = get_genes('sars_cov2.fasta', 90, True)
protein_list = get_proteins(sars2_genes)

#for i in range(len(protein_list)):
#    print(protein_list[i][3])

print('\n'.join([f'Gene {i + 1}: {protein_list[i]}' 
                 for i in range(len(protein_list))]))

Gene 1: [13, 108, '-', 'MQPSDSHTDFKVRLENRSTRDRKLVGLLPGKV']
Gene 2: [266, 13480, '+', 'MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGTCGLVEVEKGVLPQLEQPYVFIKRSDARTAPHGHVMVELVAELEGIQYGRSGETLGVLVPHVGEIPVAYRKVLLRKNGNKGAGGHSYGADLKSFDLGDELGTDPYEDFQENWNTKHSSGVTRELMRELNGGAYTRYVDNNFCGPDGYPLECIKDLLARAGKASCTLSEQLDFIDTKRGVYCCREHEHEIAWYTERSEKSYELQTPFEIKLAKKFDTFNGECPNFVFPLNSIIKTIQPRVEKKKLDGFMGRIRSVYPVASPNECNQMCLSTLMKCDHCGETSWQTGDFVKATCEFCGTENLTKEGATTCGYLPQNAVVKIYCPACHNSEVGPEHSLAEYHNESGLKTILRKGGRTIAFGGCVFSYVGCHNKCAYWVPRASANIGCNHTGVVGEGSEGLNDNLLEILQKEKVNINIVGDFKLNEEIAIILASFSASTSAFVETVKGLDYKAFKQIVESCGNFKVTKGKAKKGAWNIGEQKSILSPLYAFASEAARVVRSIFSRTLETAQNSVRVLQKAAITILDGISQYSLRLIDAMMFTSDLATNNLVVMAYITGGVVQLTSQWLTNIFGTVYEKLKPVLDWLEEKFKEGVEFLRDGWEIVKFISTCACEIVGGQIVTCAKEIKESVQTFFKLVNKFLALCADSIIIGGAKLKALNLGETFVTHSKGLYRKCVKSREETGLLMPLKAPKEIIFLEGETLPTEVLTEEVVLKTGDLQPLEQPTSEAVEAPLVGTPVCINGLMLLEIKDTEKYCALAPNMMVTNNTFTLKGGAPTKVTFGDDTVIEVQGYKSVNITFELDERIDKVLNEKCSAYTVELGTEVNEFACVVADAVIKTLQPVSELLTPLGIDLDEWSMATYYLFDESGEF